In [1]:
import lazypredict
import mat4py
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import scipy.io
import os 
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import KFold

/home/jingjinglab/cw/miniconda3/envs/Harry_env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [59]:
## 
## load data

In [64]:
list(X_train)[0]

array([ 0.96822965,  0.78817663,  0.72479129,  0.82032864,  0.92730691,
        0.50795244,  0.82515398,  0.94652056,  1.        ,  0.98032987,
        0.99550904,  0.97737923,  1.        ,  1.        ,  0.97699912,
        0.98952854,  0.7394642 ,  0.59969775,  0.92784213,  0.94694797,
        0.91149344,  0.80121812,  0.85247174,  0.76296672,  0.97651273,
        0.63809483,  0.94366512,  0.91851984,  0.7514474 ,  0.76656555,
        0.64465631,  0.80314625,  0.96465159,  0.75072608,  1.        ,
        0.74731254,  0.6661933 ,  0.89200643,  0.52239879,  0.95840181,
        0.53861451,  0.82771857,  0.95891329,  0.88437528,  1.        ,
        0.97737065,  0.67072406,  0.66955341,  0.95206264,  0.92038178,
        0.33013482, -0.15548849, -0.19372855,  0.06888124,  0.16074136,
       -0.13670387,  0.02200773, -0.01813104,  0.24927392, -0.01967013,
        0.2481965 ,  0.31118594,  0.10799357,  0.47760121,  0.01859732,
        0.45091403, -0.08825437, -0.35921554,  0.04346685, -0.05

In [14]:
import scipy.io
Connectomes = scipy.io.loadmat('../Cells2Connectomes/Connectomes.mat')
Connectome_direct = Connectomes['C_dir']


'''

Region volumes, in a 424 vector, to get connectivity density, divide
% each row in connectomes by each entry in the vector to get density. Units
% are in 200 micron per vertex voxels.

'''

CellType_volumn = mat4py.loadmat('../Cells2Connectomes/Regional_Volumes.mat')
CellType_volumn = CellType_volumn['region_vols']
Celltype_volumn =np.array([np.array(xi) for xi in CellType_volumn])
Celltype_volumn.shape

# Nomarlize by the entry

Connectome_direct_density = Connectome_direct/Celltype_volumn

Cell_type = mat4py.loadmat('../Cells2Connectomes/CellType_Maps.mat')
Cell_type = Cell_type['cellmaps']
Celltype_mtx =np.array([np.array(xi) for xi in Cell_type])
Celltype_mtx.shape

# Important : normalizing via the columns

Celltype_mtx_norm = (Celltype_mtx.max(axis=0)-Celltype_mtx) / (Celltype_mtx.max(axis=0) - Celltype_mtx.min(axis=0) )


# region names
Region_maps = mat4py.loadmat('../Cells2Connectomes/Region_Names.mat')
Region_maps = Region_maps['region_names']
Regionmaps_df = pd.DataFrame(Region_maps,columns = ['Anno1','Anno2','Anno3'])

In [65]:
Regionmaps_df[Regionmaps_df['Anno3'] =='Brainstem']

,Anno1,Anno2,Anno3
172,Visual Cortex,Visual Cortex,Brainstem
173,Brainstem,Brainstem,Brainstem
174,Brainstem,Brainstem,Brainstem
175,Brainstem,Brainstem,Brainstem
176,Brainstem,Brainstem,Brainstem
177,Brainstem,Brainstem,Brainstem
178,Brainstem,Brainstem,Brainstem
179,Brainstem,Brainstem,Brainstem
180,Brainstem,Brainstem,Brainstem
181,Brainstem,Brainstem,Brainstem


In [26]:
len(set(Regionmaps_df['Anno1']))

98

In [3]:
KF = KFold(n_splits=5, random_state=None, shuffle=True)
KF.get_n_splits(list(range(0,424)))

5

In [7]:
from random import sample

In [39]:
np.min(Celltype_mtx_norm[Celltype_mtx_norm!=0])

0.0021167075314507625

In [41]:
import math

In [46]:
np.log10(10)

1.0

In [ ]:
# random forest

# build a df to store the information

# instead use the subset, this time I am trying the whole, which might be not suitable for ML, but I just wanted a positive control
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
res_df =[]


Target = []
Dataset = []
for i in range(Celltype_mtx_norm.shape[0]):
    for j in range(Celltype_mtx_norm.shape[0]):
        #print(i)
        #print(j)
        if i == j:
            pass       
        else:

            #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
            #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
            Dataset.append(np.concatenate((Celltype_mtx_norm[i,:],Celltype_mtx_norm[j,:],
                                          Celltype_mtx_norm[i,:] - Celltype_mtx_norm[j,:], 
                                         np.log10((Celltype_mtx_norm[i,:]+ 0.0001 )/ (Celltype_mtx_norm[j,:]+0.0001)))))
            Target.append(Connectome_direct_density[i,j])
Dataset = np.stack(Dataset)
Target =np.array([np.array(xi) for xi in Target])   
    
#X_train, X_test, y_train, y_test = train_test_split(Dataset, Target,test_size=.2,random_state =123)  


kf = KFold(n_splits=10, shuffle= True)
list_r2 = []
for train_index, test_index in kf.split(Dataset):
    
    
    sub1 = train_index#), int(len(train_index))
    sub2 = test_index#), int(len(test_index)/50))
    X_train, X_test = Dataset[sub1], Dataset[sub2]
    y_train, y_test = Target[sub1], Target[sub2]
    
    
    RF_model = RandomForestRegressor(n_estimators=100,max_features = 50, warm_start = True)
    RF_model.fit(X_train, y_train)
    print('Traing fit')
    Predict = RF_model.predict(X_train)
    print(r2_score(y_train, Predict))
    print('Testing fit')
    Predict = RF_model.predict(X_test)
    print(r2_score(y_test, Predict))
    list_r2.append(r2_score(y_test, Predict))
    
    
# split into 50 %
#sub_1 = sample(list(range(X_train.shape[0])),int(X_train.shape[0]/10) )
#sub_2 = sample(list(range(X_test.shape[0])),int(X_test.shape[0]/10) )
# subset 50 percents
#Dataset_Training_sub = X_train[sub_1,:]
    
#Target_training_sub = y_train[sub_1]
    
#Dataset_Testing_sub = X_test[sub_2,:]
    
#Target_Testing_sub = y_test[sub_2]












In [58]:
list_r2

[0.21992529285145568,
 0.17196768142895924,
 0.0892550271176844,
 0.1736190164958975,
 0.3478016754367037,
 0.17150288480338083,
 0.2503509627352718,
 0.1662671732913782,
 0.27098765056574725,
 0.2559602726139455]

In [36]:
Dataset.shape

(179352, 75)

In [57]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,


In [ ]:
# random forest

# build a df to store the information

# instead use the subset, this time I am trying the whole, which might be not suitable for ML, but I just wanted a positive control
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
res_df =[]


Target = []
Dataset = []
for i in range(Celltype_mtx_norm.shape[0]):
    for j in range(Celltype_mtx_norm.shape[0]):
        #print(i)
        #print(j)
        if i == j:
            pass       
        else:

            #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
            #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
            Dataset.append(np.concatenate((Celltype_mtx_norm[i,:],Celltype_mtx_norm[j,:])))
            Target.append(Connectome_direct_density[i,j])
Dataset = np.stack(Dataset)
Target =np.array([np.array(xi) for xi in Target])   
    
#X_train, X_test, y_train, y_test = train_test_split(Dataset, Target,test_size=.2,random_state =123)  

#
kf = KFold(n_splits=5, shuffle= True)

for train_index, test_index in kf.split(Dataset):
    sub1 = train_index#), int(len(train_index)/50))
    sub2 = test_index#), int(len(test_index)/50))
    X_train, X_test = Dataset[sub1], Dataset[sub2]
    y_train, y_test = Target[sub1], Target[sub2]
  
    
    
    reg = LazyRegressor(verbose=1, predictions=True,ignore_warnings=False, custom_metric=None)
    models, predictions = reg.fit(X_train, X_test, y_train, y_test)
    print(models)
    models.to_csv('res_inproper_all.csv', sep = '\t')
    break
# split into 50 %
#sub_1 = sample(list(range(X_train.shape[0])),int(X_train.shape[0]/10) )
#sub_2 = sample(list(range(X_test.shape[0])),int(X_test.shape[0]/10) )
# subset 50 percents
#Dataset_Training_sub = X_train[sub_1,:]
    
#Target_training_sub = y_train[sub_1]
    
#Dataset_Testing_sub = X_test[sub_2,:]
    
#Target_Testing_sub = y_test[sub_2]












  2%|▏         | 1/42 [00:54<37:33, 54.95s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': -6.27437131841446, 'Adjusted R-Squared': -6.284525382231341, 'RMSE': 9.787981565238232, 'Time taken': 54.95334315299988}


  5%|▍         | 2/42 [02:11<45:05, 67.63s/it]

{'Model': 'BaggingRegressor', 'R-Squared': 0.13014442948374194, 'Adjusted R-Squared': 0.12893022572813584, 'RMSE': 3.384690670821487, 'Time taken': 76.49333715438843}


  7%|▋         | 3/42 [02:24<27:42, 42.63s/it]

...

{'Model': 'ExtraTreeRegressor', 'R-Squared': -0.6901080674689013, 'Adjusted R-Squared': -0.6924672356256139, 'RMSE': 4.717942539604872, 'Time taken': 2.9797446727752686}


 21%|██▏       | 9/42 [08:07<55:23, 100.71s/it]

{'Model': 'ExtraTreesRegressor', 'R-Squared': 0.10747014220054285, 'Adjusted R-Squared': 0.10622428812767937, 'RMSE': 3.4285207844571817, 'Time taken': 286.4595503807068}


 24%|██▍       | 10/42 [08:08<37:13, 69.81s/it]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range for family GammaDistribution


In [7]:
y_test.shape

(3587,)

In [60]:
# random forest

# build a df to store the information

# instead use the subset, this time I am trying the whole, which might be not suitable for ML, but I just wanted a positive control
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
res_df =[]


Target = []
Dataset = []
for i in range(Celltype_mtx_norm.shape[0]):
    for j in range(Celltype_mtx_norm.shape[0]):
        #print(i)
        #print(j)
        if i == j:
            pass       
        else:

            #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
            #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
            Dataset.append(np.concatenate((Celltype_mtx_norm[i,:],Celltype_mtx_norm[j,:])))
            Target.append(Connectome_direct_density[i,j])
Dataset = np.stack(Dataset)
Target =np.array([np.array(xi) for xi in Target])   
    
#X_train, X_test, y_train, y_test = train_test_split(Dataset, Target,test_size=.2,random_state =123)  

#
kf = KFold(n_splits=5, shuffle= True)

for train_index, test_index in kf.split(Dataset):
    X_train, X_test = Dataset[train_index], Dataset[test_index]
    y_train, y_test = Target[train_index], Target[test_index]
    lr = LinearRegression()
    lr.fit(X_train,y_train)


    Predict = lr.predict(X_test)
    _res = []
    _res.append('lr')

    _res.append(r2_score(y_test,Predict))
    _res.append(mean_squared_error(y_test,Predict,squared=False))
    print(_res)
    
    
#reg = LazyRegressor(verbose=1, predictions=True,ignore_warnings=False, custom_metric=None)
# split into 50 %
#sub_1 = sample(list(range(X_train.shape[0])),int(X_train.shape[0]/10) )
#sub_2 = sample(list(range(X_test.shape[0])),int(X_test.shape[0]/10) )
# subset 50 percents
#Dataset_Training_sub = X_train[sub_1,:]
    
#Target_training_sub = y_train[sub_1]
    
#Dataset_Testing_sub = X_test[sub_2,:]
    
#Target_Testing_sub = y_test[sub_2]












['RandomForestRegressor', 0.004659784996502658, 3.4779960071010225]
['RandomForestRegressor', 0.006998202035632062, 3.4902083996326403]
['RandomForestRegressor', 0.008233164737332377, 4.030816268865591]
['RandomForestRegressor', 0.0065886139845724445, 3.258057572325783]
['RandomForestRegressor', 0.007675450180245735, 3.4102158244164]


In [ ]:
# SVR

# build a df to store the information

# instead use the subset, this time I am trying the whole, which might be not suitable for ML, but I just wanted a positive control
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
res_df =[]


Target = []
Dataset = []
for i in range(Celltype_mtx_norm.shape[0]):
    for j in range(Celltype_mtx_norm.shape[0]):
        #print(i)
        #print(j)
        if i == j:
            pass       
        else:

            #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
            #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
            Dataset.append(np.concatenate((Celltype_mtx_norm[i,:],Celltype_mtx_norm[j,:])))
            Target.append(Connectome_direct_density[i,j])
Dataset = np.stack(Dataset)
Target =np.array([np.array(xi) for xi in Target])   
    
#X_train, X_test, y_train, y_test = train_test_split(Dataset, Target,test_size=.2,random_state =123)  

#
kf = KFold(n_splits=5, shuffle= True)

for train_index, test_index in kf.split(Dataset):
    print('____')
    X_train, X_test = Dataset[train_index], Dataset[test_index]
    y_train, y_test = Target[train_index], Target[test_index]
    svr = SVR()
    svr.fit(X_train,y_train)


    Predict = svr.predict(X_test)
    _res = []
    _res.append('RandomForestRegressor')

    _res.append(r2_score(y_test,Predict))
    _res.append(mean_squared_error(y_test,Predict,squared=False))
    print(_res)
    
    
#reg = LazyRegressor(verbose=1, predictions=True,ignore_warnings=False, custom_metric=None)
# split into 50 %
#sub_1 = sample(list(range(X_train.shape[0])),int(X_train.shape[0]/10) )
#sub_2 = sample(list(range(X_test.shape[0])),int(X_test.shape[0]/10) )
# subset 50 percents
#Dataset_Training_sub = X_train[sub_1,:]
    
#Target_training_sub = y_train[sub_1]
    
#Dataset_Testing_sub = X_test[sub_2,:]
    
#Target_Testing_sub = y_test[sub_2]












____


In [58]:
Dataset[1]

array([0.96822965, 0.78817663, 0.72479129, 0.82032864, 0.92730691,
       0.50795244, 0.82515398, 0.94652056, 1.        , 0.98032987,
       0.99550904, 0.97737923, 1.        , 1.        , 0.97699912,
       0.98952854, 0.7394642 , 0.59969775, 0.92784213, 0.94694797,
       0.91149344, 0.80121812, 0.85247174, 0.76296672, 0.97651273,
       0.49770912, 0.92353547, 0.98336786, 0.76338216, 0.93051494,
       0.422534  , 0.72074844, 0.91873064, 0.66188623, 0.96662907,
       0.92766184, 0.75139376, 0.97489687, 0.93668178, 0.99383866,
       0.63769894, 0.80606318, 0.91386461, 0.69750952, 0.99709589,
       0.91181692, 0.53702232, 0.72445541, 0.96431617, 0.95015779])

In [47]:
X_train.shape

(143481, 50)

In [48]:
X_test.shape

(35871, 50)

In [50]:
RF_Regmodel = RandomForestRegressor()
RF_Regmodel.fit(X_train,y_train)


Predict = RF_Regmodel.predict(X_test)
_res = []
_res.append('RandomForestRegressor')
    
_res.append(r2_score(y_test,Predict))
_res.append(mean_squared_error(y_test,Predict,squared=False))
print(_res)

['RandomForestRegressor', 0.25931770503665563, 3.115926466907411]


In [57]:
Predict = RF_Regmodel.predict(X_train[:])
r2_score(y_train[:],Predict)

0.8865365991304476

In [53]:
Target_Testing_sub.shape

(8967,)

In [34]:
Target_training_sub[:10]

array([0.        , 0.04290075, 0.05129423, 0.00204868, 0.        ,
       0.19620738, 0.        , 0.        , 0.        , 0.        ])

In [37]:
Predict[:10]

array([0.10409774, 3.59019734, 0.24403804, 0.05332101, 0.16795146,
       0.3863247 , 0.18396161, 3.47359509, 0.53028246, 1.40108984])

In [38]:
Target_Testing_sub[:10]

array([2.60533880e-04, 0.00000000e+00, 1.52469704e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.11590916e-02, 1.58809255e+00])

In [9]:
  
         
    reg = LazyRegressor(verbose=1, predictions=True,ignore_warnings=False, custom_metric=None)
    # split into 50 %
    sub_1 = sample(list(range(Dataset_Training.shape[0])),int(Dataset_Training.shape[0]/40) )
    sub_2 = sample(list(range(Dataset_Testing.shape[0])),int(Dataset_Testing.shape[0]/10) )
    # subset 50 percents
    Dataset_Training_sub = Dataset_Training[sub_1,:]
    
    Target_training_sub = Target_training[sub_1]
    
    Dataset_Testing_sub = Dataset_Testing[sub_2,:]
    
    Target_Testing_sub = Target_Testing[sub_2]
    
    #models, predictions = reg.fit(Dataset_Training_sub, Dataset_Testing_sub, Target_training_sub, Target_Testing_sub)
    #print(models)
    
    # random forest reg
    
    
    

    RF_Regmodel = RandomForestRegressor()
    RF_Regmodel.fit(Dataset_Training_sub,Target_training_sub)


    Predict = RF_Regmodel.predict(Dataset_Testing_sub)
    _res = []
    _res.append('RandomForestRegressor')
    
    _res.append(r2_score(Target_Testing_sub,Predict))
    _res.append(mean_squared_error(Target_Testing_sub,Predict,squared=False))
    res_df.append(_res)
    break 

Perform the validation, iteration:1
Build Training set
Build Testing set


In [10]:
# support vector
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


k = 0
for train_index, test_index in KF.split(list(range(0,424))):
    k = k +1
    print('Perform the validation, iteration:'+str(k))
    Targets_Train_set = Connectome_direct_density[:,train_index]
    Targets_Train_set  = Targets_Train_set[train_index,:]
    Targets_Test_set  = Connectome_direct_density[:,test_index]
    Targets_Test_set   = Targets_Test_set[test_index,:]
    
    
    Dataset_Train_set = Celltype_mtx_norm[train_index,:]
    Dataset_Test_set  = Celltype_mtx_norm[test_index,:]
    print('Build Training set')
    Target_training = []
    Dataset_Training = []
    for i in range(Dataset_Train_set.shape[0]):
        for j in range(Dataset_Train_set.shape[0]):
            #print(i)
            #print(j)
            if i == j:
                pass       
            else:

                #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
                #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
                Dataset_Training.append(np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:])))
                Target_training.append(Targets_Train_set[i,j])
    Dataset_Training = np.stack(Dataset_Training)
    Target_training =np.array([np.array(xi) for xi in Target_training])   
    
    print('Build Testing set')
    Target_Testing = []
    Dataset_Testing = []
    for i in range(Dataset_Test_set.shape[0]):
        for j in range(Dataset_Test_set.shape[0]):
            #print(i)
            #print(j)
            if i == j:
                pass       
            else:
                #print(Dataset_Test_set[i,j])

                #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
                #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
                Dataset_Testing.append(np.concatenate((Dataset_Test_set[i,:],Dataset_Test_set[j,:])))
                Target_Testing.append(Targets_Test_set[i,j])
    Dataset_Testing = np.stack(Dataset_Testing)
    Target_Testing =np.array([np.array(xi) for xi in Target_Testing])      
         
    reg = LazyRegressor(verbose=1, predictions=True,ignore_warnings=False, custom_metric=None)
    # split into 50 %
    sub_1 = sample(list(range(Dataset_Training.shape[0])),int(Dataset_Training.shape[0]/40) )
    sub_2 = sample(list(range(Dataset_Testing.shape[0])),int(Dataset_Testing.shape[0]/10) )
    # subset 50 percents
    Dataset_Training_sub = Dataset_Training[sub_1,:]
    
    Target_training_sub = Target_training[sub_1]
    
    Dataset_Testing_sub = Dataset_Testing[sub_2,:]
    
    Target_Testing_sub = Target_Testing[sub_2]
    
    #models, predictions = reg.fit(Dataset_Training_sub, Dataset_Testing_sub, Target_training_sub, Target_Testing_sub)
    #print(models)
    
    # support vector regressor
    
    
    svr_rbf = SVR()
    
    svr_rbf.fit(Dataset_Training_sub,Target_training_sub)


    Predict = svr_rbf.predict(Dataset_Testing_sub)
    _res = []
    _res.append('SVR')
    
    _res.append(r2_score(Target_Testing_sub,Predict))
    _res.append(mean_squared_error(Target_Testing_sub,Predict,squared=False))
    res_df.append(_res)
    break 

Perform the validation, iteration:1
Build Training set
Build Testing set


In [14]:
# linear regression
from sklearn.linear_model import LinearRegression



k = 0
for train_index, test_index in KF.split(list(range(0,424))):
    k = k +1
    print('Perform the validation, iteration:'+str(k))
    Targets_Train_set = Connectome_direct_density[:,train_index]
    Targets_Train_set  = Targets_Train_set[train_index,:]
    Targets_Test_set  = Connectome_direct_density[:,test_index]
    Targets_Test_set   = Targets_Test_set[test_index,:]
    
    
    Dataset_Train_set = Celltype_mtx_norm[train_index,:]
    Dataset_Test_set  = Celltype_mtx_norm[test_index,:]
    print('Build Training set')
    Target_training = []
    Dataset_Training = []
    for i in range(Dataset_Train_set.shape[0]):
        for j in range(Dataset_Train_set.shape[0]):
            #print(i)
            #print(j)
            if i == j:
                pass       
            else:

                #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
                #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
                Dataset_Training.append(np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:])))
                Target_training.append(Targets_Train_set[i,j])
    Dataset_Training = np.stack(Dataset_Training)
    Target_training =np.array([np.array(xi) for xi in Target_training])   
    
    print('Build Testing set')
    Target_Testing = []
    Dataset_Testing = []
    for i in range(Dataset_Test_set.shape[0]):
        for j in range(Dataset_Test_set.shape[0]):
            #print(i)
            #print(j)
            if i == j:
                pass       
            else:
                #print(Dataset_Test_set[i,j])

                #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
                #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
                Dataset_Testing.append(np.concatenate((Dataset_Test_set[i,:],Dataset_Test_set[j,:])))
                Target_Testing.append(Targets_Test_set[i,j])
    Dataset_Testing = np.stack(Dataset_Testing)
    Target_Testing =np.array([np.array(xi) for xi in Target_Testing])      
         
    reg = LazyRegressor(verbose=1, predictions=True,ignore_warnings=False, custom_metric=None)
    # split into 50 %
    sub_1 = sample(list(range(Dataset_Training.shape[0])),int(Dataset_Training.shape[0]/40) )
    sub_2 = sample(list(range(Dataset_Testing.shape[0])),int(Dataset_Testing.shape[0]/10) )
    # subset 50 percents
    Dataset_Training_sub = Dataset_Training[sub_1,:]
    
    Target_training_sub = Target_training[sub_1]
    
    Dataset_Testing_sub = Dataset_Testing[sub_2,:]
    
    Target_Testing_sub = Target_Testing[sub_2]
    
    #models, predictions = reg.fit(Dataset_Training_sub, Dataset_Testing_sub, Target_training_sub, Target_Testing_sub)
    #print(models)
    
    # support vector regressor
    
    
    Lr = LinearRegression()
    
    Lr.fit(Dataset_Training_sub,Target_training_sub)


    Predict = Lr.predict(Dataset_Testing_sub)
    _res = []
    _res.append('LinearRegression')
    
    _res.append(r2_score(Target_Testing_sub,Predict))
    _res.append(mean_squared_error(Target_Testing_sub,Predict,squared=False))
    res_df.append(_res)
    break 

Perform the validation, iteration:1
Build Training set
Build Testing set


In [ ]:
# linear regression
from sklearn.linear_model import LinearRegression



k = 0
for train_index, test_index in KF.split(list(range(0,424))):
    k = k +1
    print('Perform the validation, iteration:'+str(k))
    Targets_Train_set = Connectome_direct_density[:,train_index]
    Targets_Train_set  = Targets_Train_set[train_index,:]
    Targets_Test_set  = Connectome_direct_density[:,test_index]
    Targets_Test_set   = Targets_Test_set[test_index,:]
    
    
    Dataset_Train_set = Celltype_mtx_norm[train_index,:]
    Dataset_Test_set  = Celltype_mtx_norm[test_index,:]
    print('Build Training set')
    Target_training = []
    Dataset_Training = []
    for i in range(Dataset_Train_set.shape[0]):
        for j in range(Dataset_Train_set.shape[0]):
            #print(i)
            #print(j)
            if i == j:
                pass       
            else:

                #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
                #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
                Dataset_Training.append(np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:])))
                Target_training.append(Targets_Train_set[i,j])
    Dataset_Training = np.stack(Dataset_Training)
    Target_training =np.array([np.array(xi) for xi in Target_training])   
    
    print('Build Testing set')
    Target_Testing = []
    Dataset_Testing = []
    for i in range(Dataset_Test_set.shape[0]):
        for j in range(Dataset_Test_set.shape[0]):
            #print(i)
            #print(j)
            if i == j:
                pass       
            else:
                #print(Dataset_Test_set[i,j])

                #_Dataset_Training = np.concatenate((Dataset_Train_set[i,:],Dataset_Train_set[j,:]))
                #Dataset_Training = np.stack((Dataset_Training,_Dataset_Training))
                Dataset_Testing.append(np.concatenate((Dataset_Test_set[i,:],Dataset_Test_set[j,:])))
                Target_Testing.append(Targets_Test_set[i,j])
    Dataset_Testing = np.stack(Dataset_Testing)
    Target_Testing =np.array([np.array(xi) for xi in Target_Testing])      
         
    reg = LazyRegressor(verbose=1, predictions=True,ignore_warnings=False, custom_metric=None)
    # split into 50 %
    sub_1 = sample(list(range(Dataset_Training.shape[0])),int(Dataset_Training.shape[0]/40) )
    sub_2 = sample(list(range(Dataset_Testing.shape[0])),int(Dataset_Testing.shape[0]/10) )
    # subset 50 percents
    Dataset_Training_sub = Dataset_Training[sub_1,:]
    
    Target_training_sub = Target_training[sub_1]
    
    Dataset_Testing_sub = Dataset_Testing[sub_2,:]
    
    Target_Testing_sub = Target_Testing[sub_2]
    
    #models, predictions = reg.fit(Dataset_Training_sub, Dataset_Testing_sub, Target_training_sub, Target_Testing_sub)
    #print(models)
    
    # support vector regressor
    
    
    Lr = LinearRegression()
    
    Lr.fit(Dataset_Training_sub,Target_training_sub)


    Predict = Lr.predict(Dataset_Testing_sub)
    _res = []
    _res.append('LinearRegression')
    
    _res.append(r2_score(Target_Testing_sub,Predict))
    _res.append(mean_squared_error(Target_Testing_sub,Predict,squared=False))
    res_df.append(_res)
    break 

In [16]:
Result = pd.DataFrame(res_df,columns = ['Methods','R_square','RMSE'])
Result

,Methods,R_square,RMSE
0,RandomForestRegressor,-2.17,2.75
1,SVR,-0.01,2.75
2,LinearRegression,-0.09,1.73


In [ ]:
Dataset_Testing.shape

In [ ]:
from sklearn import datasets
from sklearn.utils import shuffle

In [ ]:
X_train.shape

In [ ]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(Dataset_Training, Dataset_Testing, Target_training, Target_Testing)

print(models)

In [ ]:
print(models)

In [ ]:
p=1